# Lucky Last Lab 16: visualising, interpolating, and deconvoluting data

In this lab, we'll look at a number of different ways to visualise and plot 1D and 2D data. As a final exercise, we'll combine this with the discrete Fourier transform from lab 15 to produce what looks like a miraculous "unblurring" of an image.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

We begin with a simple example to warm up. A data set called `scores.txt` is provided, containing the results of 1000 measurements of some scalar quantity.

**Use `np.loadtxt` to import these data.**

**Use the *array methods* `min`, `max`, `mean`, and `std` to determine some summary statistics of these data.** *Hint:* if you have an array variable `my_array`, the syntax of these methods is `my_array.max()` and so forth.

Of course, nothing replaces looking at these data directly. A suitable plot type would be a *histogram*, which can be made using the `plt.hist()` function. **Plot a histogram of these data.**

Hmmm, that doesn't look quite like a normal distribution. **Try increasing the number of bins** (columns), using the optional `bins=...` argument to `hist()`.

**What do you conclude about the distribution of these data?** For instance, how many *modes* (peaks of probability) are there? Do you think these data are drawn from a Gaussian distribution?

Now let's try loading some 2D data. An example is provided as `score_pairs.txt`. This array has two columns: the two points in each row correspond to two different measurements on the same sample.

**Load these data using `np.loadtxt` as before. Use `plt.hist2d`** (or, if you like, `plt.hexbin`) **to plot a 2D histogram of them. Where are most of the points concentrated?** Again, you may want to play with the number of bins.

*Hint*: Like `plot`, `hist2d` requires two arguments, one of x values and one of y. Thus you will need to pass the first and second column separately, using appropriate array slicing. Consult the `shape` of the data array if you are uncertain.

This sort of plot is appropriate when data come as pairs of measurements: for instance, the height and weight of a person, or the $x$ and $y$ coordinates where a particular plant species was observed.

Another sort of 2D data occurs when we make measurements as a function of two dependent variables – for instance, measuring the height of a mountain as a function of $x$ and $y$. The easiest case is where $x$ and $y$ form a regular grid. Data of this sort are provided as `2dgrid.txt`. 

**Load these data using `np.loadtxt`. Plot them using `plt.imshow`.**

Note two peculiarity about this plot: first, the origin is by default taken to be in the *top* left corner, which is the convention for computing but not usually helpful in science. To fix this, **use the optional argument `origin='lower'`.** Second, there is no information in the grid of data about what the $x$ and $y$ coordinates actually represent. To fix this, **use the optional `extent` argument**, given that these data represent an $x$ span of $-$0.7 to 0.35 and a $y$ span of 1.55 to 2.4. **Check your results show a peak at $(0, 2)$.**

▶ **CHECKPOINT 1**

We can use the same methods `max`, `min`, `mean`, and `std` to characterise 2D data, but there is an extra twist here: the optional `axis` argument lets us specify a single axis to look over. To see what this does, consider the following simple example:

In [ ]:
my_array = np.array([[1, 2],[3, 4]])
print(my_array)
print(my_array.max())
print(my_array.max(axis=0))
print(my_array.max(axis=1))

Use this method to **calculate the maximum value in *every row* of these data, and plot this as a function of $y$.**

*Hint*: use a suitable `linspace` to give the $y$ values. You can determine how many points are necessary using the `shape` property of an array: you'll need the first element of this array (why?).

Similarly **calculate the maximum value in *every column* of these data, and plot this as a function of $x$.**

Now let's take a different tack, by plotting a function in two dimensions. We can do this with the help of the `np.meshgrid` function. **Examine the following code and see if you can convince yourself** – looking at the documentation as well if you like – **of what this function does.**

In [ ]:
np.meshgrid((0, 1, 2), (4, 5, 6, 7))

We use this as follows, to calculate *Himmelblau's function* $f(x,y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2$:

In [ ]:
# Himmelblau's function
x, y = np.linspace(-5, 5), np.linspace(-5, 5)
X, Y = np.meshgrid(x, y)
Z = (X**2 + Y - 11)**2 + (X + Y**2 - 7)**2

The `contourf` function is used to plot "contours" joining points that have equal values of this function:

In [ ]:
plt.figure()
plt.contour(X, Y, Z)

The default settings here don't seem to be particularly helpful. The fourth, optional argument of `contourf` can either be an integer, which is interpreted as a number of contours to draw, or a list/tuple/array of values, which is interpreted as values at which to draw contours. **Find a suitable value of this argument to show clearly where the local maxima, local minima, and saddle points are in the given range $-5 \leq x, y \leq 5$. How many of each are there?**

The `contourf` function relies on the data being provided on a grid. If they are not, we first have to perform a separate interpolation step, using the `griddata` function from `scipy.interpolate`.

In [ ]:
from scipy.interpolate import griddata

Some data points drawn from this same function, presented in three columns ($x, y, f(x,y)$), are available as `nogrid.txt`:

In [ ]:
not_gridded = np.loadtxt('nogrid.txt')

The syntax of `nogrid` is a little tricky, so I've provided an example for you. The first argument is the set of $(x, y)$ points; the second is the values of $f(x,y)$, and the third is the grid to interpolate onto.

In [ ]:
gridded = griddata(not_gridded[:,0:2], not_gridded[:,2], (X, Y))

Use `contour` to **plot the newly gridded data** `gridded` on the same `X, Y` grid as before. Use `plot` to **plot the ungridded data points** `not_gridded[:,0]` against `not_gridded[:,1]` **on the same figure**. You should see that the positions of the available data are heavily influencing the contours drawn.

**Repeat the gridding step using the optional argument `method='cubic'`, a more sophisticated interpolation routine. Make a new plot of these data, again with the ungridded data points on top. Check that your results are more similar to the original function.**

▶ **CHECKPOINT 2**

## Deconvolution

Our final task will combine 2D plotting with Fourier transforms to "undo" the blurring that has made an image unreadable.

A blurred image is provided as `deconvolve.npy`. **Use `np.load`** (NB, *not* `loadtxt`) **to load this** (call it `blurred`) **and `plt.imshow` to plot it.**

We know that this image has been blurred with a Gaussian blur, so need to set up a suitable Gaussian to deconvolve with. Once again we use `meshgrid`:

In [ ]:
x = y = np.arange(-300, 300, 1)
X, Y = np.meshgrid(x, y)
sigma = 10
blurrer = np.exp(-(X**2 + Y**2)/(2*sigma**2))

**Check you understand what the code above does, then use `imshow` to plot the array `blurrer`.**

At the moment, `blurrer` follows the usual scientific convention, with the origin in the middle of the array. We need to shift it to follow the discrete Fourier transform convention, with the origin at the beginning of each axis, for which you may recall we can use the `fftshift` function:

In [ ]:
blurrer = np.fft.fftshift(blurrer)

**Use `np.fft.fft2`**, the 2D Fourier transform function, **to transform both `blurrer` and `blurred`, calling your results `ftblurrer` and `ftblurred`.**

Rather than plotting these results directly, we will plot the regions where there is significant signal:

In [ ]:
plt.figure()
plt.imshow(abs(ftblurrer) > 1e-10)
plt.figure()
plt.imshow(abs(ftblurred) > 1e-10)

You will see that only low-frequency components (*i.e.*, the corners of the plot) are present in both FTs.
**Adjust `sigma` from the `blurrer` calculation until these regions above $10^{-7}$ are about the same size, but slightly larger in `blurrer` than `blurred`.** At this point, we are using all the available information.

We're now ready to deconvolve. We divide through by `ftblurrer` *only in those regions where this value is not so small that it will cause problems*:

In [ ]:
ftblurred[abs(ftblurrer) > 1e-10] /= ftblurrer[abs(ftblurrer) > 1e-10]

Now **use `np.fft.ifft2` to inverse Fourier transform `ftblurred`, and `imshow` to plot it.**

*Hint:* If your result is still blurred, `sigma` is too small; if it looks completely unlike the input, `sigma` is too large.

▶ **CHECKPOINT 3**

**Extension:** Experiment with different `sigma` values. What happens when this value is too small or too large? Why?